In [9]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange

url = "https://www.law.go.kr/DRF/lawSearch.do?OC=jennie0715&target=prec&type=XML"
response = urlopen(url).read()
xtree = ET.fromstring(response)

totalCnt = int(xtree.find('totalCnt').text)

page = 1
rows = []
for i in trange(int(totalCnt / 20)):
    try:
        items = xtree[5:]
    except:
        break
        
    for node in items:
        판례일련번호 = node.find('판례일련번호').text
        사건명 = node.find('사건명').text
        사건번호 = node.find('사건번호').text
        선고일자 = node.find('선고일자').text
        법원명 = node.find('법원명').text
        사건종류명 = node.find('사건종류명').text
        사건종류코드 = node.find('사건종류코드').text
        판결유형 = node.find('판결유형').text
        선고 = node.find('선고').text
        판례상세링크 = node.find('판례상세링크').text

        rows.append({'판례일련번호': 판례일련번호,
                    '사건명': 사건명,
                    '사건번호': 사건번호,
                    '선고일자': 선고일자,
                    '법원명': 법원명,
                    '사건종류명': 사건종류명,
                    '사건종류코드': 사건종류코드,
                    '판결유형': 판결유형,
                    '선고': 선고,
                    '판례상세링크': 판례상세링크})
    page += 1
    url = "https://www.law.go.kr/DRF/lawSearch.do?OC=jennie0715&target=prec&type=XML&page={}".format(page)
    response = urlopen(url).read()
    xtree = ET.fromstring(response)
cases = pd.DataFrame(rows)
cases.to_csv('./cases.csv', index=False)

100%|██████████| 4314/4314 [10:29<00:00,  6.86it/s]


### 판례 목록 중 사건명에서 사기를 포함하는 것만 선별
#### 군사기밀, 수사기관은 제외

In [10]:
import numpy as numpy
import pandas as pd

case_list = pd.read_csv('./cases.csv')
contents = ['판시사항', '판결요지', '참조조문', '참조판례', '판례내용']

In [11]:
case_list_사기 = []
for i in range(len(case_list)):
    사건명 = case_list['사건명'].iloc[i]
    if isinstance(사건명, str):  # Check if 사건명 is a string
        if '사기' in 사건명:
            if '군사기밀' not in 사건명 and '수사기관' not in 사건명:
                case_list_사기.append(case_list.iloc[i])

In [12]:
case_list_사기 = pd.DataFrame(case_list_사기).reset_index(drop=True)

In [13]:
case_list_사기

,판례일련번호,사건명,사건번호,선고일자,법원명,사건종류명,사건종류코드,판결유형,선고,판례상세링크
0,238021,특정경제범죄가중처벌등에관한법률위반(사기)·사기,2023도12424,2023.11.16,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...
1,237943,의료법위반·특정경제범죄가중처벌등에관한법률위반(사기)방조·의료법위반방조[적법하게 개설...,2022도90,2023.10.26,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...
2,236477,의료법위반·특정경제범죄가중처벌등에관한법률위반(사기)[비의료인이 개설자격을 위반하여 ...,2020도6492,2023.08.18,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...
3,236823,특정경제범죄가중처벌등에관한법률위반(사기)·의료법위반,2017도1807,2023.07.17,대법원,형사,400102.0,전원합의체 판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...
4,237903,사기·사문서위조·위조사문서행사·주민등록법위반·범죄수익은닉의규제및처벌등에관한법률위반,2023노357,2023.07.13,서울북부지방법원,형사,400102.0,판결 : 상고,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...
...,...,...,...,...,...,...,...,...,...,...
2263,215295,"사기,횡령",4287형상76,1955.02.25,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...
2264,215359,"사기,업무횡령,법령제193호위반피고",4286형상25,1954.10.05,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...
2265,86037,사기피고,4287형상119,1954.06.07,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...
2266,86031,사기피고,4287형상208,1954.04.15,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...


### 판례 전문 text 파일로 저장

In [14]:
import os
import re
def remove_tag(content):
    cleaned_text = re.sub('<.*?>', '', content)
    return cleaned_text

for content in contents:
    os.makedirs('./판례/{}'.format(content), exist_ok=True)

for i in trange(len(case_list_사기)):
    url = "https://www.law.go.kr"
    link = case_list_사기.loc[i]['판례상세링크'].replace('HTML', 'XML')
    url += link
    response = urlopen(url).read()
    xtree = ET.fromstring(response)

    for content in contents:
        text = xtree.find(content).text
        # 내용이 존재하지 않는 경우 None 타입이 반환되기 때문에 이를 처리해줌
        if text is None:
            text = '내용없음'
        else:
            text = remove_tag(text)
        file = './판례/' + content + '/' + xtree.find('판례정보일련번호').text + '.txt'
        with open(file, 'w') as c:
            c.write(text)
            c.close()

100%|██████████| 2268/2268 [04:53<00:00,  7.73it/s]


### case_list 데이터프레임에 붙이기

In [15]:

def remove_tag(content):
    cleaned_text = re.sub('<.*?>', '', content)
    return cleaned_text

case_list_사기[contents] = 0


In [18]:
case_list_사기

,판례일련번호,사건명,사건번호,선고일자,법원명,사건종류명,사건종류코드,판결유형,선고,판례상세링크,판시사항,판결요지,참조조문,참조판례,판례내용
0,238021,특정경제범죄가중처벌등에관한법률위반(사기)·사기,2023도12424,2023.11.16,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...,"유사수신행위를 금지·처벌하는 유사수신행위의 규제에 관한 법률 제6조 제1항, 제3...","유사수신행위의 규제에 관한 법률(이하 ‘유사수신행위법’이라 한다) 제6조 제1항, ...","유사수신행위의 규제에 관한 법률 제3조, 제6조 제1항, 형법 제37조, 제347조...",내용없음,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 장용배【배상신청인】 배...
1,237943,의료법위반·특정경제범죄가중처벌등에관한법률위반(사기)방조·의료법위반방조[적법하게 개설...,2022도90,2023.10.26,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...,[1] 의료법인 명의로 개설된 의료기관을 실질적으로 비의료인이 개설·운영하였다고...,[1] 의료법인 명의로 개설된 의료기관을 실질적으로 비의료인이 개설·운영하였다고...,[1] 구 의료법(2019. 8. 27. 법률 제16555호로 개정되기 전의 것)...,[1] 대법원 2023. 7. 17. 선고 2017도1807 전원합의체 판결(공2...,【피 고 인】 피고인 1 외 5인【상 고 인】 피고인들【변 호 인】 법무법인 우리하...
2,236477,의료법위반·특정경제범죄가중처벌등에관한법률위반(사기)[비의료인이 개설자격을 위반하여 ...,2020도6492,2023.08.18,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...,[1] 의료법인 명의로 개설된 의료기관을 실질적으로 비의료인이 개설·운영하였다고...,[1] 의료법인 명의로 개설된 의료기관을 실질적으로 의료인의 자격이 없는 일반인...,[1] 구 의료법(2019. 4. 23. 법률 제16375호로 개정되기 전의 것)...,[1] 대법원 2023. 7. 17. 선고 2017도1807 전원합의체 판결(공2...,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 법무법인(유한) 해송 담당변호...
3,236823,특정경제범죄가중처벌등에관한법률위반(사기)·의료법위반,2017도1807,2023.07.17,대법원,형사,400102.0,전원합의체 판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...,[1] 의료인 개인 명의로 개설된 의료기관이 구 의료법 제33조 제2항에서 정한...,[1] 대법원은 의료인 개인 명의로 개설된 의료기관이 실질적으로 의료인의 자격이...,[1] 구 의료법(2015. 12. 29. 법률 제13658호로 개정되기 전의 것...,[1][2] 대법원 2011. 10. 27. 선고 2009도2629 판결(공201...,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 법무법인(유) 동인 외 1인【...
4,237903,사기·사문서위조·위조사문서행사·주민등록법위반·범죄수익은닉의규제및처벌등에관한법률위반,2023노357,2023.07.13,서울북부지방법원,형사,400102.0,판결 : 상고,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...,피고인이 성명 불상의 전화금융사기 조직원의 제안을 받고 현금수거책 역할을 담당하...,피고인이 성명 불상의 전화금융사기 조직원의 제안을 받고 현금수거책 역할을 담당하...,"형법 제30조, 제231조, 제234조, 제347조 제1항, 주민등록법 제37조 제...",내용없음,【피 고 인】 피고인【항 소 인】 검사【검 사】 이정성 외 1인【변 호 인】 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263,215295,"사기,횡령",4287형상76,1955.02.25,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...,0,0,0,0,0
2264,215359,"사기,업무횡령,법령제193호위반피고",4286형상25,1954.10.05,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...,0,0,0,0,0
2265,86037,사기피고,4287형상119,1954.06.07,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...,0,0,0,0,0
2266,86031,사기피고,4287형상208,1954.04.15,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...,0,0,0,0,0


In [17]:
os

<module 'os' (frozen)>

In [19]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
import os
import re

for content in contents:
    os.makedirs('./판례/{}'.format(content), exist_ok=True)

for i in trange(len(case_list_사기)):
    url = "https://www.law.go.kr"
    link = case_list_사기.loc[i]['판례상세링크'].replace('HTML', 'XML')
    url += link
    response = urlopen(url).read()
    xtree = ET.fromstring(response)

    for content in contents:
        text = xtree.find(content).text
        # 내용이 존재하지 않는 경우 None 타입이 반환되기 때문에 이를 처리해줌
        if text is None:
            text = '내용없음'
        else:
            text = remove_tag(text)
        case_list_사기[content].loc[i] = text

  0%|          | 0/2268 [00:00<?, ?it/s]

100%|██████████| 2268/2268 [04:45<00:00,  7.96it/s]


In [20]:
case_list_사기

,판례일련번호,사건명,사건번호,선고일자,법원명,사건종류명,사건종류코드,판결유형,선고,판례상세링크,판시사항,판결요지,참조조문,참조판례,판례내용
0,238021,특정경제범죄가중처벌등에관한법률위반(사기)·사기,2023도12424,2023.11.16,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...,"유사수신행위를 금지·처벌하는 유사수신행위의 규제에 관한 법률 제6조 제1항, 제3...","유사수신행위의 규제에 관한 법률(이하 ‘유사수신행위법’이라 한다) 제6조 제1항, ...","유사수신행위의 규제에 관한 법률 제3조, 제6조 제1항, 형법 제37조, 제347조...",내용없음,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 장용배【배상신청인】 배...
1,237943,의료법위반·특정경제범죄가중처벌등에관한법률위반(사기)방조·의료법위반방조[적법하게 개설...,2022도90,2023.10.26,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...,[1] 의료법인 명의로 개설된 의료기관을 실질적으로 비의료인이 개설·운영하였다고...,[1] 의료법인 명의로 개설된 의료기관을 실질적으로 비의료인이 개설·운영하였다고...,[1] 구 의료법(2019. 8. 27. 법률 제16555호로 개정되기 전의 것)...,[1] 대법원 2023. 7. 17. 선고 2017도1807 전원합의체 판결(공2...,【피 고 인】 피고인 1 외 5인【상 고 인】 피고인들【변 호 인】 법무법인 우리하...
2,236477,의료법위반·특정경제범죄가중처벌등에관한법률위반(사기)[비의료인이 개설자격을 위반하여 ...,2020도6492,2023.08.18,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...,[1] 의료법인 명의로 개설된 의료기관을 실질적으로 비의료인이 개설·운영하였다고...,[1] 의료법인 명의로 개설된 의료기관을 실질적으로 의료인의 자격이 없는 일반인...,[1] 구 의료법(2019. 4. 23. 법률 제16375호로 개정되기 전의 것)...,[1] 대법원 2023. 7. 17. 선고 2017도1807 전원합의체 판결(공2...,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 법무법인(유한) 해송 담당변호...
3,236823,특정경제범죄가중처벌등에관한법률위반(사기)·의료법위반,2017도1807,2023.07.17,대법원,형사,400102.0,전원합의체 판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...,[1] 의료인 개인 명의로 개설된 의료기관이 구 의료법 제33조 제2항에서 정한...,[1] 대법원은 의료인 개인 명의로 개설된 의료기관이 실질적으로 의료인의 자격이...,[1] 구 의료법(2015. 12. 29. 법률 제13658호로 개정되기 전의 것...,[1][2] 대법원 2011. 10. 27. 선고 2009도2629 판결(공201...,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 법무법인(유) 동인 외 1인【...
4,237903,사기·사문서위조·위조사문서행사·주민등록법위반·범죄수익은닉의규제및처벌등에관한법률위반,2023노357,2023.07.13,서울북부지방법원,형사,400102.0,판결 : 상고,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...,피고인이 성명 불상의 전화금융사기 조직원의 제안을 받고 현금수거책 역할을 담당하...,피고인이 성명 불상의 전화금융사기 조직원의 제안을 받고 현금수거책 역할을 담당하...,"형법 제30조, 제231조, 제234조, 제347조 제1항, 주민등록법 제37조 제...",내용없음,【피 고 인】 피고인【항 소 인】 검사【검 사】 이정성 외 1인【변 호 인】 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263,215295,"사기,횡령",4287형상76,1955.02.25,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...,신구 사기죄와 형의 경중,신 형법의 사기죄의 형은 벌금의 선택형이 있으므로 구 형법의 사기죄의 형보다 경하...,"형법 제347조, 구 형법 제24조",내용없음,"【상고인, 피고인】 피고인【원심판결】 제1심 광주지방법원 목포지원, 제2심 광주고등..."
2264,215359,"사기,업무횡령,법령제193호위반피고",4286형상25,1954.10.05,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...,공판준비 또는 공판기일에 피고인 또는 피고인 아닌 자의 진술이 공소사실에 부합되는...,공판준비 또는 공판기일에 피고인 또는 피고인 아닌 자의 진술이 공소사실에 부합되는...,내용없음,내용없음,【상 고 인】 검사 주운화【원 심】 대구고등법원【주 문】 원판결을 ...
2265,86037,사기피고,4287형상119,1954.06.07,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...,증거취사와 사실인정,증거에 의하지 아니한 사실인정은 증거법칙에 위배한 위법이라 아니할 수 없다,내용없음,내용없음,"【상고인, 피고인】 【검 사】 이호용【원심판결】 \n제1심 부산지방법원, ..."
2266,86031,사기피고,4287형상208,1954.04.15,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=jennie0715&target=prec&I...,행위자의 예상과 범의의 성부,범죄행위시의 행위자의 결과에 대한 예상과 그 발생한 결과가 상위된 경우라 할지라도...,"형법 제13조, \n 제347조",내용없음,"【상 고 인】 검사【원심판결】 \n제1심 대구지방법원 안동지원, 제2심 대구고등..."


In [21]:
case_list_사기['판례내용'][case_list_사기['판례내용']=='내용없음'].count() # 판결요지는 928개가 내용없음. 무조건 판례내용을 써야할 듯

0

In [23]:
case_list_사기.to_csv("fraud.csv", header=True, encoding="cp949", index=True)